# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Jacopo De Stefani - __[Jacopo.De.Stefani@ulb.ac.be](mailto:Jacopo.De.Stefani@ulb.ac.be)__
### Théo Verhelst - __[Theo.Verhelst@ulb.ac.be](mailto:Theo.Verhelst@ulb.ac.be)__
### Gianluca Bontempi - __[gbonte@ulb.ac.be](mailto:gbonte@ulb.ac.be)__

## TP 3 - Predictions: Tree-based methods

####  March 23, 2021

#### Materials originally developed by *Bertrand Lebichot, Jacopo De Stefani and Gianluca Bontempi*

## Preliminaries

### Supervised learning

The process of supervised learning involves the presence of an entity (the learner, also called prediction model), whose goal is to learn the mapping between inputs and outputs in a given problem.

A supervised learning problem can formulated as follows:

\begin{equation*}
 y = m(\mathbf{x})  
\end{equation*}

where:
- $y$ represents the output variable (also called target)
- $\mathbf{x}$ represents the vector of inputs (also called features).
- $m$ is the (unknown) mapping between input and outputs.

In the majority of the supervised learning problems, the mapping $m$ between input and outputs is unknown and needs to be estimated on basis of the available input/output observation pairs $(\mathbf{x}_i,y_i)$.

## Classification vs regression

Both classification and regression are sub-fields of *supervised learning*. In the two cases, we have predictive variables $\mathbf{x}$ and a target variable $y$. 
The main difference betweet the two type of problems is the type of the target variabile:

- In classification, $y$ is a discrete variable; i.e $y \in \{C_1,\cdots,C_k\}$
- In regression, $y$ is a continuous variable; i.e $y \in \mathbb{R}$

In this practical, in order to better understand the classification process, we will tackle the simplest classification case, with $k=2$ possible output classes (called binary classification).

The goal of our approach is to be able to learn the mapping between the input features and the predefined discrete output classes, in order to be able to perform an automatic classification, based on the available data. 

Let's illustrate the problem with a simple example :
* Knowing the two abstract features $X_1$ and $X_2$, and $n=6$ samples for both class (the red class and the blue class), can you predict the label for the green sample ?


In [ ]:
n = 6

# blue class data
X1 = rnorm(n, mean = +2, sd = 1)
X2 = rnorm(n, mean = +1, sd = 1)
SamplesBLUE = data.frame(X1, X2)
gBLUE = c(mean(X1),mean(X2))

# red class data
X1 = rnorm(n, mean = -2, sd = 1)
X2 = rnorm(n, mean = -1, sd = 1)
SamplesRED = data.frame(X1, X2)
gRED = c(mean(X1),mean(X2))

# draw scatterplot
plot(SamplesBLUE$X1,SamplesBLUE$X2,col="blue",main="Scatterplot and decision boundary",xlab="X1",ylab="X2",xlim=c(-5,5),ylim=c(-5,5))
points(SamplesRED$X1,SamplesRED$X2,col="red")
points(0,0,col="green")

# draw centroid of classes
points(gBLUE[1],gBLUE[2],col="blue",pch = 4)
points(gRED[1],gRED[2],col="red",pch = 4)

# draw decision boundary
midgg = c(gRED[1]+abs(gBLUE[1]-gRED[1])/2,gRED[2]+abs(gBLUE[2]-gRED[2])/2)
slope = -(gBLUE[1]-gRED[1])/(gBLUE[2]-gRED[2])
abline(a=midgg[2]-slope*midgg[1],b=slope)


## Classification metrics

### Confusion Matrix 

A confusion matrix $\mathbf{C}$ is a $k \times k$ matrix containing the classifications statistics of a given classifier.
$c_{ij}$ containes the number of times that a sample belonging to the actual class $j$ has been predicted as belonging to class $i$.

In our two-class setting, the confusion matrix (reporting the number of actual class / predicted class) has four entries:
 
|            | Actual Negative   | Actual Positive  |
|:----------:|-------------------|-----------------|
|**Classified Negative** | $T_N$ (True Negative) | $F_N$ (False Negative)| 
|**Classified Positive** | $F_P$ (False Positive) | $T_P$ (True Positive) |

Ideally, if the classifier didn't make any mistake, the confusion matrix $\mathbf{C}$ should be diagonal.
By looking at the off-diagonal elements, we can understand which kind of mistakes the classifier is making (e.g. Actual Negative -> Predicted Positive, Actual Positive -> Predicted Negative).

Additionally, the confusion matrix allows to compute the total number of elements classified negative $\hat{N_N}$, classified positive $\hat{N_P}$, actual negative $N_N$, actual positive $N_P$, as well as the total number of samples $N$.

\begin{align*}
\hat{N_N} = T_N + F_N & & & \hat{N_P} = T_P + F_P \\ 
N_N = T_N + F_P & & & N_P = T_P + F_N \\
\end{align*}

\begin{equation}
N = T_N + F_N + T_P + F_P 
\end{equation}

The quantities in the confusion matrix are used to define different accuracy measures, such as:

#### Accuracy

\begin{equation}
A = \frac{T_P + T_N}{N} = \frac{T_P + T_N}{F_P + F_N + T_P + T_N} 
\end{equation}

The accuracy represents the ratio between the number of correctly classified samples (False ...) and the total number of samples.

#### Misclassification Rate

\begin{equation}
ER = \frac{F_P + F_N}{N} = \frac{F_P + F_N}{F_P + F_N + T_P + T_N} 
\end{equation}

The misclassification rate represents the number of total classification mistakes (False ...) over the total number of samples.
It can be shown that ER=1-A.

#### Balanced Error Rate
\begin{equation}
BER = \frac{1}{2}(\frac{F_P}{N_N} + \frac{F_N}{N_P}) = \frac{1}{2}(\frac{F_P}{T_N + F_P} + \frac{F_N}{T_P + F_N}) 
\end{equation}

The balanced error rate is computed as an average of the error for each classes.
For unbalanced classification problems (i.e. $N_N \neq N_P$ the total number of samples belonging to the negative and positive classes is different).

#### Sensitivity and specificity
\begin{align}
SE = \frac{T_P}{N_P} = \frac{T_P}{T_P + F_N} & & & 0 \geq SE \leq 1 
\end{align}

The sensitivity (also called recall) is a measure defined as the ratio between the correctly classified positive samples over the total number of positive samples (i.e True Positive rate). It measures the impact of false negatives on the classification process.

\begin{align}
SP = \frac{T_N}{N_N} = \frac{T_N}{F_P + T_N}  & & & 0 \geq SP \leq 1
\end{align}

The specificity is a measure defined as the ratio between the correctly classified negative samples over the total number of negative samples (i.e True Negative rate). It measures the impact of false positive on the classification process.

The need for different accuracy measures arises from the fact that the impact of a certain type of error (e.g. Actual Negative -> Predicted Positive, Actual Positive -> Predicted Negative) might be greatly different, according to the context in which the classification problem is performed (for example medical diagnosis, fraud detection).


## Dataset

The chosen database is about spam detection. Details can be found here :
https://archive.ics.uci.edu/ml/datasets/spambase

To import the dataset, just use package ''kernlab''

`install.packages("kernlab")`

In case of doubts, always remember that R can provide you with the documentation of a function package/using the following syntax:
`? name_of_function` or `help(name_of_function)` 

In [ ]:
library("kernlab")
library("tree")
library("rpart")
data(spam)

## Exercise 0 - Exploratory Data Anaylsis 

You first have to explore the dataset before going to classification. This step can be long but is actually very important. In particular, obtain or observe the following interesting elements :
* Number of rows and columns
* Is there any missing values ?
* Spam prior
* Name and basic statistics for each variables
* Histogram (or other relevant plot) per class (spam vs non-spam)

### Number of rows and columns

In [ ]:
dim(spam)

### Presence of missing values

In [ ]:
# No missing values -> dim(na.omit(dataset)) == dim(dataset)
dim(na.omit(spam))

In [ ]:
# 2nd Check
apply(is.na(spam),2,any)

### Structure of the dataset

In [ ]:
head(spam)

In [ ]:
summary(spam)

### Spam Prior

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
table(spam$type)
plot(spam$type)

### Histogram (or other relevant) plots of the variables with respect to the target

In [ ]:
library("ggplot2"); library("GGally")
ggpairs(spam[,c(21,23,7,57,58)],aes(colour = type))

## Classification trees 

### Model structure

A classification tree is a model employing a tree-based structure in order to perform classification of the target variable.
The tree structure contains internal nodes, and terminal nodes (often called leaves).

Each internal node is associated to a feature in the input vector $\mathbf{x}$, and performs a partition of the input space according to the value of the associated feature.
Each terminal node is associated to a specific output label $C_i$.
Internal nodes can be concatenated with each other in order to specify additional partitions of the input space.

By combining these definitions, we can notice that the terminal nodes partitions the input space into mutually exclusive regions (in a divide-and-conquer fashion). 

The classification operation is performed by traversing the tree from the root node until one of the terminal nodes is reached.
At each intermediate internal node, a decision is made according to the value associated to the corresponding feature (for example, for the i-th feature $x_i \geq 0$).
According to the outcome of this decision, a different path in the tree is taken.
The tree traversal is stopped once a terminal node is reached, and the output of the classifier correspond to the label associated to the terminal node (e.g. $C_0$).

Hence, by simply looking at the tree structure, we can easily understand the sequence of operations (i.e. splits) that yielded to the classification decision, improving interpretability of the model.

### Learning procedure

As an analogy to their biological equivalent, the learning procedure for a decision tree has two steps known as *tree growing* and *tree pruning*.

* During *tree growing*, an iterative, exhaustive search is performed to find the successive splits, selecting the one that minimizes a certain cost function.
  
  Here you can find two examples of commonly used cost functions, with $p_i$ representing the probability of selecting a sample of class $i$ in the input space subset defined by the considered split:

    * The **Gini Impurity** (used in CART) : how often a randomly chosen element would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. It is minimum (zero) for class-homogeneous nodes.
    
    \begin{equation}
         Gini = \sum_{i=1}^C p_i * (1-p_i) = 1-\sum_{i=1}^C p_i^2\\
    \end{equation}

    * The **information gain** (IG, used in ID3,C4.5) : based on the concept of entropy and information content from information theory. Basically the IG is the entropy of the parent node minus the weighted sum of the children node entropy.

    \begin{equation}
         IG = -\sum_{i=1}^C p_i * log_2 (p_i) - \sum_{a} p(a) \sum_{i=1}^C Pr(i|a) * log_2 (Pr(i|a))  \\
    \end{equation}
   

* During *tree pruning*, some branches of the decision tree are removed, based on a complexity based measure of the tree performance, in order to avoid overfitting.


An animated version of decision trees can be found here : http://www.r2d3.us/visual-intro-to-machine-learning-part-1/



## Exercise 1 - Classification with decision trees

On the given dataset, our classification task will be to discriminated between spam and non-spam samples ($k=2$ classes - binary classificiation task).
Among the difference performance metric presented, we will be using the misclassification rate. 

In order to study the generalization capabilities of our learning model, we will be dividing our dataset in two part :

* *training set* : Sample of data that we will be using to perform the training of the model, presenting to the model the $(\mathbf{x}_i,y_i)$ pairs.
* *test set* : Sample of data, unseen by the model, used to assess the predictions made by the training model.

In order to obtain a more statistically sound estimation of the performance of the model, this procedure is often repeated several times, with different training-test splits (k-fold cross-validation).

For this exercise, we will ask you to:

* First, obtain the performance of a DT on the spam dataset using a 50%/50% training-test partition. 
* Check the importance of each feature using the obtained DT using *rpart* or *tree*.
* Then plot the obtained tree using packages *rpart.plot*.
* Finally, implement a 10-fold cross-validation for assessing your DT.

**N.B.** The selected classifier actually outputs an estimation of the conditional probabilities $p(y_i=C_0|\mathbf{x})$ and $p(y_i=C_1|\mathbf{x_i})$, $C_0 and C_1$ being respectively "nonspam" and "spam". In order to transform the conditional probability into the corresponding class, we will define a threshold $t=0.5$. If $p(y_i=C_1|\mathbf{x_i}) > t$ than the sample $\mathbf{x_i}$ will be affected to the class "spam", otherwise it will be affected to the "non spam" category
 


## Exercise 2 - Classification : Alternative metrics (ROC and AUC)

In the previous exercise we selected an arbitrary threshold $t=0.5$ in order to determine the output classes.
The confusion matrix and all the related performance measures depends on the choice of the aforementioned threshold. 
Usually the choice of a threshold is related to Type I error and Type II errors (cf. Section 5.13 and 10.3 of the syllabus) that we are ready to accept in a stochastic setting.

In order to avoid conditioning our assessment on a specific threshold, it is interesting to assess the overall accuracy for all possible thresholds. This is possible by plotting curves, like the Receiver Operating Characteristic (ROC).

The ROC curve is a plot in which, for every threshold in the range $[0,1]$, the false positive rate f the corresponding classifier (1-specificity, on the horizontal axis) against the true positive rate (i.e. sensitivity or power, on the vertical axis). In other terms, ROC visualises the probability of detection vs. the probability of false alarm. Each point on the curve correspond to a different threshold used in the classifier.

The ideal ROC curve would be composed of two straight lines from $(FPR=0,TPR=0)$ -> $(FPR=0,TPR=1)$ -> $(TPR=1,FPR=1)$. 
The worst possible ROC curve would be represented by the bissextrix line on the FPR-TPR plane.
In this case, for each threshold, we would have $T_P=N_P=F_P=N_N$ , i.e. the same proportion of true positive and false positive. 
In other terms, this classifier would not separate the classes at all.
In practice, real-life classification rules produce ROC curves which lie between these two extremes. 

A common way to summarise a ROC curve, is to compute the area under the curve (AUC), by curve integration. 
By measuring the AUC of different classifiers, we have a compact way
to compare classifiers without setting a specific threshold.

For the classifier defined in Exercise 1, you need to:

- Plot the ROC curve for $t \in \{0,0.05,0.1,\cdots,1\}$
- Compare the ROC curve with the ideal and worst classifier
- Compute the corresponding AUC using a trapezoidal approximation
    - *Example:* Given a curve composed by three points ($x_1,y_1$),($x_2,y_2$),($x_3,y_3$), the AUC is computed as the sum of the trapezoid defined by the points 1-2 and the trapezoid defined by the points 2-3:    $AUC = AUC_{12} + AUC_{23} = \frac{(x_2-x_1)*(y_1+y_2)}{2} + \frac{(x_3-x_2)*(y_2+y_3)}{2}$  

Based on: 
- https://mmuratarat.github.io/2019-10-01/how-to-compute-AUC-plot-ROC-by-hand
- https://stackoverflow.com/questions/4954507/calculate-the-area-under-a-curve

## Exercise 3 - Ensemble learning : random forest

Random Forests (RF) is an ensemble learning technique proposed by Breiman which combines bagging and random feature selection by using a large number of non pruned decision trees.
Hence, Each individual tree is trained on a different subset of samples (due to bagging), as well as a different subset of features (due to random feature selection)
The random feature selection for every tree allows to decorrelate the predictions of the different trees. 
The aggregation of decorrelated classifiers allows to reduce the variance of the final prediction.
In the case of classification, the aggregation is performed by majority vote (i.e. the class that is predicted by the largest number of individual classifiers is selected as the prediction).
In the case of regression, the aggregation is performed by performing the average of the individual predictions.

For this exercise you need to:
- Implement a random forest with five trees from your (simple data partition) DT. Do not use the *randomForest* package.
- Test the difference in accuracy with 8 and 20 features per tree.
- Implement a 10-fold cross-validation for assessing your DT. Plot the performances according to the number of trained trees. 
- Bonus: Repeat the previous steps using the  *randomForest* package.

Here is an implementation helper:

* The number of trees is a compromise between training time and performance.
* Each tree is built on a subset of features, a good heuristic being the squared root of the total number of features, drawn randomly.
* For a new sample, the predicted class is simply the mean of predicted probabilities on all trained trees.


## Exercise 4 - Ensemble learning : Boosting (Ada Boost)

Boosting is an additional generic ensemble technique which attempts to boost the accuracy of any given learning algorithm.
The focus of boosting methods is to produce a series of weak learners in order to produce a powerful combination. 
A weak learner is a learner that has accuracy only slightly better than chance.
In boosting, the training set used for each member of the series is chosen based on the performance of the earlier classifier(s) in the series. 
Examples that are incorrectly predicted by previous classifiers in the series are chosen more often than examples that were correctly predicted.
Thus Boosting attempts to produce new classifiers that are better able to predict examples for which the current ensemble’s performance is poor. 
Unlike Bagging (employed in Random Forests), the resampling of the training set is dependent on the performance of the earlier classifiers, which prevents a parallel implementation of the ensembling procedure.

For this exercise you need to:
- Implement a boosting ensemble of 15 decision trees using Ada Boost (cf. 11.3.1)
- The initial sampling probability for each sample $i$ is $w_i = \frac{1}{N}$ (with $1 \leq i \leq N$, $N$ being the number of samples in train set). 
- After the first tree, $w_i$ is adjusted according to the classification performances:
    \begin{equation} w_i = w_i \begin{cases} e^{-\alpha_j} & \text{if sample $i$ correctly classified} \\ e^{\alpha_j} & \text{if sample $i$ incorrectly classified} \end{cases} \end{equation}
    \begin{equation} alpha_j = \log(\frac{1-ER_j}{ER_j}) \end{equation}
    \begin{equation} E_j = \frac{\sum_{i=1}^{N} w_{i} I\left(y_{i} \neq h_{j}\left(x_{i}\right)\right)}{\sum_{i=1}^{N} w_{i}} \end{equation}
      
- The prediction of the boosting ensemble is defined as a linear combination (weighted by $\alpha_j$) of the outputs of the different trees:
    \begin{equation} h_{boo} = sign\left( \sum_{j=1}^{m} = \alpha_j*h_j(x)\right) \end{equation}

    

Here is an implementation helper (multiple variants exist) :

- The number of trees is a compromise between training time and performance. 
- The trees are usually depth-forced : add *control=tree.control(Ntrain,mincut=10)* as a parameter to *tree()*.
- For this exercice, we will use the *tree* package. It is therefore easier to recode the spam labels as -1/+1.